# Spot Finding for diffraction/background sorting

used to move files into diffraction and potential backgrounds folder

In [ ]:
# import relevant libraries
import numpy as np
import matplotlib.pyplot as plt
import h5py
import os
import matplotlib as mpl

In [ ]:
# read in all nxs file names in given directory
# directory needs "/" not "\" and a "/" on the end
directory = "C:/directory/to/your/data/"
nxs_count = 0
nxs_files = []

# Count the number of .nxs files in directory
for files in os.listdir(directory):
    filename = os.fsdecode(files)
    if filename.endswith(".nxs"):
        nxs_files.append(filename)
        nxs_count += 1
print('File count:', nxs_count)

In [ ]:
processing_folder = "processing_info/"
processing_path = os.path.join(directory,processing_folder)


if not os.path.exists(processing_path):
    os.makedirs(processing_path)

In [ ]:
# read in all frames in directory into a numpy array
frame_sized = np.zeros((nxs_count, 1441, 1440))
a = 0

while a < nxs_count:
    with h5py.File(directory+nxs_files[a], 'r') as dat:
        nxs_frame = np.array(dat["/entry1/pixium_hdf/data"][()][:]) 
        frame_sized[a] = nxs_frame.reshape(nxs_frame.shape[1:])
        a += 1

In [ ]:
# select value of the "5th" (or other, based on pixel_intensity_no) most intense pixel
# print those pixel intensities

pixel_intensity_no = 5

flat_intensities = np.zeros((nxs_count, 2075040))
pixel_values = np.zeros((nxs_count))
b = 0

while b < nxs_count:
    flat_intensities[b] = frame_sized[b].flatten()
    flat_intensities[b].sort()
    pixel_values[b] = flat_intensities[b][-pixel_intensity_no]
    b += 1
    
print(pixel_values)

In [ ]:
#used to rename files for plot, need to change [138:-1] depending on you directory location string length

datasets_name = directory[138:-1]
modified_datasets_name = datasets_name.replace("/", "_")
plot_name = modified_datasets_name + "_" + str(pixel_intensity_no) +"th_I_plot.png"
plot_directory = directory + plot_name

print(modified_datasets_name)
print(plot_name)
print(plot_directory)

In [ ]:
# plot graph of frame name vs pixel intensities
plt.rcParams["figure.figsize"]=18,10
plt.title(datasets_name + " Intensity plot", fontsize=24)
plt.bar(nxs_files, pixel_values, width=0.8, align='center')
plt.xticks(range(len(nxs_files)), nxs_files, rotation='vertical')
plt.xlabel('Frame name', fontsize=15)
plt.ylabel('Intensity of '+str(pixel_intensity_no)+'th most intense pixel',fontsize=15)
plt.savefig(processing_path + plot_name)
plt.show()


In [ ]:
# read in hdf file names
hdf_count = 0
hdf_files = []


for files2 in os.listdir(directory):
    filename2 = os.fsdecode(files2)
    if filename2.endswith(".hdf"):
        hdf_files.append(filename2)
        hdf_count += 1
        
names_values = np.array([nxs_files, pixel_values, hdf_files])

In [ ]:
# threshold potential backgrounds/diffraction frames with spots_threshold
# list the number of potential backgrounds and names of these

# change spots threshold to desired cut off for diffraction/background

spots_threshold = 800

diffraction_frames = []
diffraction_hdf = []
background_frames = []
background_hdf = []
c = 0

while c < nxs_count:
    if float(names_values[:, c][1]) > spots_threshold:
        diffraction_frames.append(names_values[:, c][0])
        diffraction_hdf.append(names_values[:,c][2])
        c += 1
    else:
        background_frames.append(names_values[:, c][0])
        background_hdf.append(names_values[:,c][2])
        c += 1

print('Number of potential background frames:',len(background_frames))
background_frames

In [ ]:
# name and make the directory for the potential backgrounds to be put into

backgrounds_folder = "Potential backgrounds/"
bg_path = os.path.join(directory,backgrounds_folder)


if not os.path.exists(bg_path):
    os.makedirs(bg_path)

In [ ]:
# move the nxs and hdf files for the potential backgrounds into the potential backgrounds directory

for files in os.listdir(directory):
    filename = os.fsdecode(files)
    if filename in background_frames:
        os.replace(os.path.join(directory,filename), os.path.join(bg_path,filename))
        
    elif filename in background_hdf:
        os.replace(os.path.join(directory,filename), os.path.join(bg_path,filename))

# Now check background/diffraction frame in DAWN and sort into appropriate folder

DAWN download at https://dawnsci.org/

In [ ]:
diff_count = 0

for files in os.listdir(directory):
    filename = os.fsdecode(files)
    if filename.endswith(".nxs"):
        diff_count += 1
print('Sorted diffraction frames:', diff_count)


bg_count = 0

for files in os.listdir(bg_path):
    filename = os.fsdecode(files)
    if filename.endswith(".nxs"):
        bg_count += 1
print('Sorted background frames:', bg_count)

In [ ]:
hit_rate = round((diff_count/nxs_count * 100), 1)
hit_rate_text = str(hit_rate) + "%"
print("Hit rate: " + hit_rate_text)

In [ ]:
output_file_path = processing_path + modified_datasets_name+ "_hit_rates.txt"

with open(output_file_path, 'w') as file:
    file.write(f"Datasets location: {directory}\n\n")
    file.write(f"Total frames:   {nxs_count}\n\n")
    file.write(f"Pixel intensity number: {pixel_intensity_no}\n")
    file.write(f"Spots threshold: {spots_threshold}\n")
    file.write(f"Number of thresholded background frames: {str(len(background_frames))}\n\n")
    file.write(f"Frames with diffraction:    {diff_count} \n")
    file.write(f"Sorted background frames:    {bg_count} \n\n")
    file.write(f"Hit rate:    {hit_rate_text} \n\n")